# Modeling explosive volcanism on Mercury
Here we employ the thermodanymic magmatic gas model of Iacovino (2015), in python beta as "TAVERNmag". Experimental results detailed in the main text as well as solubility limits are used as constraints to determine which magmatic volatile compositions could feasibly have produced the observed pyroclastic deposits on Mercury.


First we import the necessary libraries, including the model of Iacovino (2015) in python beta "TAVERNmag". We also define a status_bar, which we can use to display the progress of long computations.

In [1]:
import sys
 
# setting path
sys.path.append('..')

%pip install plotly
%pip install scikit-image
%pip install kaleido

import tavernmag as tv
import plotly.figure_factory as ff
import plotly.express as px
import numpy as np
import sys
import pickle
import copy

# Define a status bar for long calls
def status_bar(percent, btext=None, barLen=20):
    """
    Prints an updating status bar to the terminal or jupyter notebook.

    Parameters
    ----------
    percent: float
        Percent value of progress from 0 to 1

    btext: string
        Any extra text to display next to status bar

    barLen: int
        Length of bar to print
    """
    sys.stdout.write("\r")
    sys.stdout.write("[{:<{}}] {:.0f}%".format("=" * int(barLen * percent),
                                               barLen, percent * 100))
    if btext is not None:
        sys.stdout.write(" " + str(btext))
    if percent == 1.0:
        sys.stdout.write("\n")
    sys.stdout.flush()

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


## Define necessary constants

In [2]:
# Fluid species considered
fluid_species_names = ['CO', 'CO2', 'H2', 'H2O', 'H2S', 'O2', 'S2', 'SO2']

# Molecular Weights
molecularWeight = tv.core.oxideMass

# Specific Heat Rations (C_P/C_V) at 1600 K and 1 atm
specificHeatRatios = {'CO': 1.306,
                      'CO2': 1.166,
                      'H2': 1.39,
                      'H2O': 1.21,
                      'H2S': 1.19,
                      'O2': 1,
                      'S2': 1.29,
                      'SO2': 1.18}

# Other constants
g = 3.7 # m/s2, gravitational acceleration due to gravity on Mercury
R = 8.3145 # J/mol-K, gas constant
T = 1600 # degrees K, temperature

# Define deposit radius in meters
deposit_radius = 110800

## Generate bulk gas compositions
Here we generate all potential bulk gas compositions for a C-O-H-S fluid in terms of H$_{tot}$, C$_{tot}$, and S$_{tot}$ in steps of 5 mol% such that they evenly cover the entire H-C-S binary. These are then translated into H$_2$O$_{tot}$, CO$_{2 tot}$, and S$_{tot}$ for input into TAVERNmag. These will be speciated properly according to the system $f$O$_2$, pressure, and temperature in the next step.

### Generic gas compositions
Here we create a generic list of all H$_{tot}$, C$_{tot}$, and S$_{tot}$ values ranging from 1 to 100 in steps of 5. We are creating compositions in this space since that is how we will plot them up later.

In [3]:
# define a function to generate an evenly spaced list
def sums(length, total_sum, step):
    """Generate a list of tuples where each tuple has length 
    number of values and a sum of total_sum. Step is the step
    size."""
    if length == 1:
        yield (total_sum,)
    else:
        for value in range(total_sum + 1):
            for permutation in sums(length - 1, total_sum - value, step):
                remainder = value % step
                if remainder == 0:
                    yield (value,) + permutation

# create a dict with keys Htot, Ctot, and Stot, each ranging from 1-100 in steps of 5
bulk_comp_list_simp = []
for l in list(sums(3,100,5)):
    bulk_comp_list_simp.append({'Htot': l[0], 'Ctot': l[1], 'Stot': l[2]})

# create new dict to be converted to moles and wt%
bulk_comp_list_moles = []
bulk_comp_list_wtper = []
for l in list(sums(3,100,5)):
    bulk_comp_list_moles.append({'Htot': l[0], 'Ctot': l[1], 'Stot': l[2]})
    bulk_comp_list_wtper.append({'Htot': l[0], 'Ctot': l[1], 'Stot': l[2]})

### Convert gas compositions to H$_2$O$_{tot}$, CO$_{2,tot}$, S$_{tot}$ in moles
We need to convert our generic compositions from H$_{tot}$, C$_{tot}$, and S$_{tot}$ to H$_2$O$_{tot}$, CO$_{2,tot}$, and S$_{tot}$ (e.g., where all H species are given as H$_2$O) since these are the inputs required by TAVERNmag. This is done separately so that the H-C-S values for each set of gas compositions cover the entire ternary space evenly. The below compositions will be speciated properly in the next step, when we apply the TAVERNmag model.

Below we compute the gas compositions in terms of mol%.

In [4]:
# convert to H2Otot, CO2tot, Stot
# in terms of moles
for comp in bulk_comp_list_moles:
    comp['Htot'] /= 0.6666
    comp['Ctot'] *=3
    
    # normalize
    comp_sum = comp['Htot']+comp['Ctot']+comp['Stot']
    comp['Htot'] *= 100/comp_sum
    comp['Ctot'] *= 100/comp_sum
    comp['Stot'] *= 100/comp_sum

    # rename
    comp['H2O'] = comp.pop('Htot')
    comp['CO2'] = comp.pop('Ctot')
    comp['S'] = comp.pop('Stot')

### Convert gas compositions to H2Otot, CO2tot, Stot in wt%
Here we compute the gas compositions as above, but in terms of wt%.

In [5]:
# in terms of wtpercent
for comp in bulk_comp_list_wtper:
    comp['Htot'] *= (molecularWeight['H2O']/molecularWeight['H2'])
    comp['Ctot'] *= (molecularWeight['CO2']/molecularWeight['C'])

    # normalize
    comp_sum = comp['Htot']+comp['Ctot']+comp['Stot']
    comp['Htot'] *= 100/comp_sum
    comp['Ctot'] *= 100/comp_sum
    comp['Stot'] *= 100/comp_sum

    # rename
    comp['H2O'] = comp.pop('Htot')
    comp['CO2'] = comp.pop('Ctot')
    comp['S'] = comp.pop('Stot')

Optionally, we can print the three dicts that we made to check their values.

In [6]:
for i in range(len(bulk_comp_list_simp)):
    print(bulk_comp_list_simp[i])
    print(bulk_comp_list_wtper[i])
    #print(bulk_comp_list_moles[i])
    print('\n')

{'Htot': 0, 'Ctot': 0, 'Stot': 100}
{'H2O': 0.0, 'CO2': 0.0, 'S': 100.0}


{'Htot': 0, 'Ctot': 5, 'Stot': 95}
{'H2O': 0.0, 'CO2': 16.167468672544654, 'S': 83.83253132745534}


{'Htot': 0, 'Ctot': 10, 'Stot': 90}
{'H2O': 0.0, 'CO2': 28.933712804795068, 'S': 71.06628719520494}


{'Htot': 0, 'Ctot': 15, 'Stot': 85}
{'H2O': 0.0, 'CO2': 39.26987712213637, 'S': 60.730122877863636}


{'Htot': 0, 'Ctot': 20, 'Stot': 80}
{'H2O': 0.0, 'CO2': 47.809518015747486, 'S': 52.19048198425252}


{'Htot': 0, 'Ctot': 25, 'Stot': 75}
{'H2O': 0.0, 'CO2': 54.98356489897193, 'S': 45.016435101028094}


{'Htot': 0, 'Ctot': 30, 'Stot': 70}
{'H2O': 0.0, 'CO2': 61.09532916652977, 'S': 38.90467083347022}


{'Htot': 0, 'Ctot': 35, 'Stot': 65}
{'H2O': 0.0, 'CO2': 66.36448962331845, 'S': 33.63551037668154}


{'Htot': 0, 'Ctot': 40, 'Stot': 60}
{'H2O': 0.0, 'CO2': 70.95405881883498, 'S': 29.04594118116501}


{'Htot': 0, 'Ctot': 45, 'Stot': 55}
{'H2O': 0.0, 'CO2': 74.98754751208664, 'S': 25.012452487913375}


{'Htot': 0,

## Use TAVERNmag to speciate each bulk gas composition at 1 bar, 1400 °C, and $f$O$_2$ of IW-4.5
Below we have used pickle files to save having to recompute these lists every time the notebook is run. A pickle file is simply a way to save some variable to permanent memory such that it can be reimported for use later. The below code checks to see if there is a pickle file saved. If so, it simply loads the values computed previously. If not, it computes the values. The pickle files have the extension ".p" and are saved in the same location as this notebook file. Deleting them and rerunning this notebook will force the code to calculate these values anew.

### Some notes on model optimization and computation time
As the python implementation of Iacovino (2015; TAVERNmag) is still in beta, the choice of precisely which minimization functions to use to solve various functions within the code is not yet finalized. In particular, the calculation of fugacities requires the minimization of a set of two equations with two unknowns (eqn. 9 in Iacovino, 2015) to solve for $f$H$_2$ and $f$S$_2$. In general, the choice of minimization function does not significantly change the calculated fugacities, but it can effect model instabilities. For example, the use of scipy.optimize's least_squares minimization routine results in model instabilities on the H$_{tot}$-S$_{tot}$ binary at approximately H$_{tot}$ = 70 mol% and S$_{tot}$ = 30 mol% and extending also toward the C vertex of the teranary. Instead here we employ the GEKKO optimization library, which results in no instabilities but is extremely computationally intensive. Where scipy.optimize may take <1 minute to complete the two calls below, GEKKO may take 20 minutes.

GEKKO is currently the default behavior, but passing `opt='leastsq'` to the `tv.calculate_speciation()` function will result in the use of the scipy.optimize minimization routine.

### Speciate molar gas compositions
We continue below to compute molar and wt% gas compositions separately. This is so that when we recast these into H$_{tot}$, C$_{tot}$, and S$_{tot}$ space, the speciated compositions and other calculated values corresponding to each composition covers the entire H-C-S ternary evenly.

In [7]:
# in terms of moles
try:
    speciated_gas_list_moles_wtfrac = pickle.load(open("speciated_gas_list_moles_wtfrac_new.p", "rb"))
except: 
    speciated_gas_list_moles = []
    iterno = 0
    for gascomp in bulk_comp_list_moles:
        iterno += 1
        # create a tv.MagmaticFluid() object
        fluid_obj = tv.MagmaticFluid(gascomp, units='molpercent', default_units='wtpercent')
        speciated_gas_list_moles.append(tv.calculate_speciation(sample=fluid_obj, pressure=1, temperature=1400,
                                                                fO2_buffer="IW", fO2_delta=-4.5).result)
        percent = iterno/len(bulk_comp_list_moles)
        status_bar(percent)

    # convert from wt% to wt frac
    speciated_gas_list_moles_wtfrac = []
    for gascomp in speciated_gas_list_moles:
        speciated_gas_list_moles_wtfrac.append({k: v/100 for k, v in gascomp.items()})

    # Save a pickle file of this calculation, since it takes soooo long...
    pickle.dump(speciated_gas_list_moles_wtfrac, open("speciated_gas_list_moles_wtfrac_new.p", "wb"))

### Speciate wt% gas compositions

In [8]:
# in terms of wtpercent
try:
    speciated_gas_list_wtper_wtfrac = pickle.load(open("speciated_gas_list_wtper_wtfrac_new.p", "rb"))
except:
    speciated_gas_list_wtper = []
    iterno = 0
    for gascomp in bulk_comp_list_wtper:
        iterno += 1
        #create a tv.MagmaticFluid() object
        fluid_obj = tv.MagmaticFluid(gascomp, units='wtpercent', default_units='wtpercent')
        speciated_gas_list_wtper.append(tv.calculate_speciation(sample=fluid_obj, pressure=1, temperature=1400,
                                                                fO2_buffer="IW", fO2_delta=-4.5).result)
        percent = iterno/len(bulk_comp_list_wtper)
        status_bar(percent)

    # convert from wt% to wt frac
    speciated_gas_list_wtper_wtfrac = []
    for gascomp in speciated_gas_list_wtper:
        speciated_gas_list_wtper_wtfrac.append({k: v/100 for k, v in gascomp.items()})

    # Save a pickle file of this calculation, since it takes soooo long...
    pickle.dump(speciated_gas_list_wtper_wtfrac, open("speciated_gas_list_wtper_wtfrac_new.p", "wb"))

## Calculate sum total of released volatiles for set of pyroclastic deposit radii

Using equation 5 from the main text to solve for $n_{Tot}$ the volatile weight fraction of a mixed gas:

$n_{Tot}$ = $\frac{gX}{20RT}$$\cdot$ $\sum_{}^{}W_{i}$ ( $\frac{m_{i}(\gamma_{i}-1)}{\gamma_{i}}$ )

Where $g$ is the gravitational acceleration on Mercury, $X$ is the maximum distance that particles can be ejected on an airless body in meters (radius of the pyroclastic deposit), $R$ is the gas constant (8.3145 J-mol$^{-1}$-K$^{-1}$), $T$ is the temperature in K, $m_i$ is the molecular weight of gas $i$, $\gamma_i$ is the specific heat ratio of gas $i$, and $W_i$ is the weight fraction of gas $i$ in the total gas.

Constants used here are defined above in cell "Define necessary constants". The value of $X$ can be varied to calculate ternary diagrams for any given pyroclastic deposit radius.


In [9]:
X = deposit_radius #deposit radius in meters as defined by user above

### Calculate $n_{Tot}$ for molar compositions (in wt%)

- Input: Speciated gases generated from molar compositions, in weight fraction
- Output: $n_{Tot}$ values for each composition in wt%

In [10]:
# using moles composition space
nTot_list_from_moles = []
for mixed_gas in speciated_gas_list_moles_wtfrac:
    nsum = sum([mixed_gas[species]*(molecularWeight[species]*(specificHeatRatios[species]-1)/specificHeatRatios[species]) for species in fluid_species_names])
    nTot_list_from_moles.append((g*X)/(20*R*T) * nsum)

### Calculate $n_{tot}$ for wt% compositions (in wt%)

- Input: Speciated gases generated from wt% compositions, in weight fraction
- Output: $n_{Tot}$ values for each composition in wt%

In [11]:
# using wtpercent composition space
nTot_list_from_wtper = []
for mixed_gas in speciated_gas_list_wtper_wtfrac:
    nsum = sum([mixed_gas[species]*(molecularWeight[species]*(specificHeatRatios[species]-1)/specificHeatRatios[species]) for species in fluid_species_names])
    nTot_list_from_wtper.append((g*X)/(20*R*T) * nsum)

### Convert molar $n_{tot}$ values from wt% to mol%

Next, in order to plot $n_{Tot}$ values both in terms of wt% and mol%, we must convert the output for our molar composition from wt% into mol%. To do this, we need to consider the bulk silicate melt composition. Here we use the composition of S-free Sil-3 from the main text.

In [12]:
# for moles composition space
# Recalculate nTot values as moles using S-free Sil-3 as bulk composition of melt

silicate_composition_wtpercent = {'SiO2': 54.05,
                                  'TiO2': 1.26,
                                  'Al2O3': 12.60,
                                  'Cr2O3': 0.74,
                                  'FeO': 3.40,
                                  'MnO': 0.65,
                                  'MgO': 14.25,
                                  'CaO': 5.23,
                                  'K2O': 0.25,
                                  'Na2O': 6.81}
nTot_list_as_moles = []
for i, mixed_gas in enumerate(speciated_gas_list_moles_wtfrac):
    bulk_silicate_w_volatiles = silicate_composition_wtpercent
    for species in fluid_species_names:
        bulk_silicate_w_volatiles[species] = nTot_list_from_moles[i]*speciated_gas_list_moles_wtfrac[i][species]
    mol_prop_ox = {k: v/molecularWeight[k] for k, v in bulk_silicate_w_volatiles.items()}
    bulk_silicate_w_volatiles_moles = {k: v/sum(mol_prop_ox.values()) for k, v in mol_prop_ox.items()}
    nTot_as_moles = 100*sum(v for k, v in bulk_silicate_w_volatiles_moles.items() if k in fluid_species_names)
    nTot_list_as_moles.append(nTot_as_moles)

### Plot results (as gas)
Here we generate Figures 5a and 5b in the main text. These show the amount of gas in mol% or wt%, respectively, required to produce pyroclastic deposits of a given radius (as defined by variable $X$ above) at 1 bar, 1400 °C and an $f$O$_2$ of IW-4.5. Note that in these plots, we consider all gas species.

In [13]:
# molpercent composition space
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

fig = ff.create_ternary_contour(np.array([Htot, Ctot, Stot], dtype=float), np.array(nTot_list_as_moles, dtype=float),
                                pole_labels=['$H_{tot}$', '$C_{tot}$', '$S_{tot}$'],
                                interp_mode='cartesian',
                                ncontours=20,
                                colorscale='Viridis',
                                showscale=True,
                                showmarkers=False,
                                title='As gas, in mol%')
fig.show()
fig.write_image("as_gas_molper.svg")

# wtpercent composition space
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

fig = ff.create_ternary_contour(np.array([Htot, Ctot, Stot], dtype=float), np.array(nTot_list_from_wtper, dtype=float),
                                pole_labels=['$H_{tot}$', '$C_{tot}$', '$S_{tot}$'],
                                interp_mode='cartesian',
                                ncontours=20,
                                colorscale='Viridis',
                                showscale=True,
                                showmarkers=False,
                                title='As gas, in wt%')
fig.show()
fig.write_image("as_gas_wtper.svg")

## Calculate dissolved C, H, S in magma (no solubility limits, no smelting)
Next, we can calculate the concentration of dissolved C, H, and S in a magma that would generate the gas compositions plotted in the previous section. We first do this calculation without regard to solubility limits.

First, we convert our speciated gases back into C$_{tot}$, H$_{tot}$, and S$_{tot}$ and then sum these. We will do this in wt% composition space.

### Calculate dissolved volatiles for wt% compositions (in wt%)

In [14]:
#using wtpercent composition space
C_dissolved = []
H_dissolved = []
S_dissolved = []
tot_dissolved = []
for i in range(len(nTot_list_from_wtper)):
    C_from_CO = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['CO']*molecularWeight['C']/molecularWeight['CO']
    C_from_CO2 = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['CO2']*molecularWeight['C']/molecularWeight['CO2']
    H_from_H2 = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['H2']*molecularWeight['H']/molecularWeight['H2']
    H_from_H2O = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['H2O']*molecularWeight['H']/molecularWeight['H2O']
    H_from_H2S = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['H2S']*molecularWeight['H']/molecularWeight['H2S']
    S_from_S2 = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['S2']*molecularWeight['S']/molecularWeight['S2']
    S_from_SO2 = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['SO2']*molecularWeight['S']/molecularWeight['SO2']
    S_from_H2S = nTot_list_from_wtper[i]*speciated_gas_list_wtper_wtfrac[i]['H2S']*molecularWeight['S']/molecularWeight['H2S']
    C_dissolved.append(C_from_CO + C_from_CO2)
    H_dissolved.append(H_from_H2 + H_from_H2O + H_from_H2S)
    S_dissolved.append(S_from_S2 + S_from_SO2 + S_from_H2S)
    tot_dissolved.append(C_from_CO + C_from_CO2 + H_from_H2 + H_from_H2O + H_from_H2S + S_from_S2 + S_from_SO2 + S_from_H2S)

### Convert dissolved values into mol% space (in mol%)
Next, we can convert these dissolved volatile concentrations from wt% to mol%. To do this, we need to consider the bulk silicate melt composition. Here we use the composition of S-free Sil-3 from the main text.

In [15]:
# Recalculate tot_dissolved values as moles using S-free Sil-3 as bulk composition of melt
silicate_composition_wtpercent_diss = {'SiO2': 54.05,
                                  'TiO2': 1.26,
                                  'Al2O3': 12.60,
                                  'Cr2O3': 0.74,
                                  'FeO': 3.40,
                                  'MnO': 0.65,
                                  'MgO': 14.25,
                                  'CaO': 5.23,
                                  'K2O': 0.25,
                                  'Na2O': 6.81}

tot_diss_as_moles = []
for i in range(len(tot_dissolved)):
    bulk_with_volatiles_diss = silicate_composition_wtpercent_diss
    bulk_with_volatiles_diss['C'] = C_dissolved[i]
    bulk_with_volatiles_diss['H'] = H_dissolved[i]
    bulk_with_volatiles_diss['S'] = S_dissolved[i]
    
    #convert to mol%
    mol_prop_ox = {k: v/molecularWeight[k] for k, v in bulk_with_volatiles_diss.items()}
    bulk_with_volatiles_diss_moles = {k: 100*v/sum(mol_prop_ox.values()) for k, v in mol_prop_ox.items()}
    tot_diss_as_moles.append(bulk_with_volatiles_diss_moles['C'] + bulk_with_volatiles_diss_moles['H'] + 
                             bulk_with_volatiles_diss_moles['S'])

### Plot results (dissolved volatiles, no solubility limits, no smelting)

Here we generate Figures 5c and 5d in the main text. These demonstrate the corresponding amount of dissolved H, C, and/or S in mol% or wt%, respectively, required to produce the same deposits modeled above assuming volatiles sourced only from the magma (not constrained by solubility limits). Note that in these plots, we consider dissolved H, C, and S only whereas the previous plots considered all gas species. For example, here all C species are computed in as the concentration of C (as is typically done when reporting dissolved volatile contents, since particular complexing in the melt may not be known). In the previous plots, CO and CO$_2$ are both considered in the gas.

In [16]:
# molpercent composition space
# bulk_comp_list
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

# use nTot_list a function

fig = ff.create_ternary_contour(np.array([Htot, Ctot, Stot], dtype=float), np.array(tot_diss_as_moles, dtype=float),
                                pole_labels=['$H_{diss}$', '$C_{diss}$', '$S_{diss}$'],
                                interp_mode='cartesian',
                                ncontours=20,
                                colorscale='YlOrRd',
                                showscale=True,
                                showmarkers=False,
                                title='Dissolved volatiles, in mol% (without smelting)')
fig.show()
fig.write_image("diss_molper_nosmelt.svg")

# wtpercent composition space
# bulk_comp_list
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

# use nTot_list a function

fig = ff.create_ternary_contour(np.array([Htot, Ctot, Stot], dtype=float), np.array(tot_dissolved, dtype=float),
                                pole_labels=['$H_{diss}$', '$C_{diss}$', '$S_{diss}$'],
                                interp_mode='cartesian',
                                ncontours=20,
                                colorscale='YlOrRd',
                                showscale=True,
                                showmarkers=False,
                                title='Dissolved volatiles, in wt% (without smelting)')
fig.show()
fig.write_image("diss_wtper_nosmelt.svg")

## Calculate dissolved C, H, S in magma _in addition to smelting_ (no solubility limits)
The above ternaries illustrate the dissolved volatile concentrations corresponding to the gas releases needed to generate pyroclastic deposits of a given radius. That assumes no solubility limits on the magma. It also assumes that all of the gas released originated from dissolved volatiles in a melt and does not consider the additional volatiles that may be sourced via smelting. 

Our experiments indicate a maximum of 5 wt% FeO smelted in the melt, which would require the smelting of <1 wt% graphite and result in ~2 wt% pure CO gas or an equivalent deposit radius of ~21,000 m. Given smelting of 5 wt% FeO in the melt, we can calculate the mass of _additional_ volatiles needed to produce deposits larger than 21,000 m.

### First, calculate $n_{tot}$ for volatiles in addition to smelting

In [17]:
# using wtpercent composition space, only volatiles needed in addition to smelting
nTot_list_from_wtper_minus_smelting = []
for mixed_gas in speciated_gas_list_wtper_wtfrac:
    nsum = sum([mixed_gas[species]*(molecularWeight[species]*(specificHeatRatios[species]-1)/specificHeatRatios[species]) for species in fluid_species_names])
    X_wSmelt = X-21000
    if X_wSmelt <0:
        X_wSmelt = 0
    nTot_list_from_wtper_minus_smelting.append((g*(X_wSmelt))/(20*R*T) * nsum)

### Second, calculate dissolved H, C, and S in magma required in addition to smelting

In [18]:
#using wtpercent compsition space, in addition to volatiles from smelting
C_dissolved_wSmelt = []
H_dissolved_wSmelt = []
S_dissolved_wSmelt = []
tot_dissolved_wSmelt = []
for i in range(len(nTot_list_from_wtper_minus_smelting)):
    C_from_CO = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['CO']*molecularWeight['C']/molecularWeight['CO']
    C_from_CO2 = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['CO2']*molecularWeight['C']/molecularWeight['CO2']
    H_from_H2 = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['H2']*molecularWeight['H']/molecularWeight['H2']
    H_from_H2O = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['H2O']*molecularWeight['H']/molecularWeight['H2O']
    H_from_H2S = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['H2S']*molecularWeight['H']/molecularWeight['H2S']
    S_from_S2 = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['S2']*molecularWeight['S']/molecularWeight['S2']
    S_from_SO2 = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['SO2']*molecularWeight['S']/molecularWeight['SO2']
    S_from_H2S = nTot_list_from_wtper_minus_smelting[i]*speciated_gas_list_wtper_wtfrac[i]['H2S']*molecularWeight['S']/molecularWeight['H2S']
    C_dissolved_wSmelt.append(C_from_CO + C_from_CO2)
    H_dissolved_wSmelt.append(H_from_H2 + H_from_H2O + H_from_H2S)
    S_dissolved_wSmelt.append(S_from_S2 + S_from_SO2 + S_from_H2S)
    tot_dissolved_wSmelt.append(C_from_CO + C_from_CO2 + H_from_H2 + H_from_H2O + H_from_H2S + S_from_S2 + S_from_SO2 + S_from_H2S)

### Finally, plot ternary results for dissolved volatiles (without solubility limits) in addition to smelting

Here we plot Figure 5e from the main text. This figure demonstrates the amount of dissolved H, C, and/or S in wt% required to produce deposits greater than 21,000 m in radius in addition to ~2 wt% CO produced via the smelting of 5 wt% FeO.

In [19]:
# wtpercent composition space in addition to smelting
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

fig = ff.create_ternary_contour(np.array([Htot, Ctot, Stot], dtype=float), np.array(tot_dissolved_wSmelt, dtype=float),
                                pole_labels=['$H_{diss}$', '$C_{diss}$', '$S_{diss}$'],
                                interp_mode='cartesian',
                                ncontours=20,
                                colorscale='Cividis',
                                showscale=True,
                                showmarkers=False,
                                title='Dissolved volatiles, in wt% (with smelting of 5 wt% FeO)')
fig.show()
fig.write_image("diss_wtper_smelt.svg")

## Calculate plausible dissolved C, H, S in magma (with solubility limits)

Taken alone, the above calculations do not consider volatile solubility. That is, we set no a priori limit to the amount of H, C, or S that a mercurian magma could hold. Within this set of all possible gas compositions, we can use solubility constraints to narrow down the range of plausible dissolved volatile contents necessary to drive the largest pyroclastic deposits on Mercury. Here we assume maximum dissolved volatile concentrations in reduced mercurian magmas of ~1000 ppm total C (Li et al., 2017), ~1–5 wt% total S (Namur et al., 2016), and ~3000 ppm total H (Hirschmann et al., 2012). H and C maxima are taken based on the highest H and C reported in solubility experiments performed on basalts at reducing conditions (Li et al., 2017; Hirschmann et al., 2012). The S solubility is extremely dependent upon temperature. The Sil-3 composition used in our experiments (representing _Borealis Planitia_) at 1340 °C and 2.5 GPa can contain up to ~1 wt% S (modeled with Namur et al., 2016). More refractory compositions like that of the High Magnesium Region require higher temperatures to be molten and therefor have higher modeled S solubilities, up to around 5 wt% S. Here, we take the upper bound of 5 wt% as an estimate of the maximum S carrying capacity for any mercurian melt.

Solubility limits used here:
- C = 1000 ppm (Li et al., 2017)
- H = 3000 ppm (Hirschmann et al., 2017)
- S = 5 wt% (Namur et al., 2016)

### Plot all plausible dissolved volatile concentrations with or withour smelting
Only deposits with radii ≤60 km produce any plausible dissolved volatile concentrations. Change the value of $X$ above to see how these plots change with deposit radius.

In [20]:
# mol composition space, without smelting
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

plausible_pts_H = []
plausible_pts_C = []
plausible_pts_S = []
for i in range(len(Htot)):
    H_diss = tot_diss_as_moles[i] * Htot[i]/100
    C_diss = tot_diss_as_moles[i] * Ctot[i]/100
    S_diss = tot_diss_as_moles[i] * Stot[i]/100
    if H_diss <= 14.25 and C_diss <= 0.4 and S_diss <= 7.48:
        plausible_pts_H.append(Htot[i])
        plausible_pts_C.append(Ctot[i])
        plausible_pts_S.append(Stot[i])

fig = px.scatter_ternary(tot_dissolved, a=plausible_pts_H, b=plausible_pts_C, c=plausible_pts_S)
print("Mol% no smelting")
fig.show()
fig.write_image("diss_mol_no_smelting_plausible.svg")

# wtpercent composition space, without smelting
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

plausible_pts_H = []
plausible_pts_C = []
plausible_pts_S = []
for i in range(len(Htot)):
    H_diss = tot_dissolved[i] * Htot[i]/100
    C_diss = tot_dissolved[i] * Ctot[i]/100
    S_diss = tot_dissolved[i] * Stot[i]/100
    if H_diss <= 0.3 and C_diss <= 0.1 and S_diss <= 5.0:
        plausible_pts_H.append(Htot[i])
        plausible_pts_C.append(Ctot[i])
        plausible_pts_S.append(Stot[i])
plausible_pts_H

fig = px.scatter_ternary(tot_dissolved, a=plausible_pts_H, b=plausible_pts_C, c=plausible_pts_S)
print("Wt% no smelting")
fig.show()
fig.write_image("diss_no_smelting_plausible.svg")

# wtpercent composition space, with smelting
Htot = [x['Htot'] for x in bulk_comp_list_simp]
Ctot = [x['Ctot'] for x in bulk_comp_list_simp]
Stot = [x['Stot'] for x in bulk_comp_list_simp]

plausible_pts_H = []
plausible_pts_C = []
plausible_pts_S = []
for i in range(len(Htot)):
    H_diss = tot_dissolved_wSmelt[i] * Htot[i]/100
    C_diss = tot_dissolved_wSmelt[i] * Ctot[i]/100
    S_diss = tot_dissolved_wSmelt[i] * Stot[i]/100
    if H_diss <= 0.3 and C_diss <= 0.1 and S_diss <= 5.0:
        plausible_pts_H.append(Htot[i])
        plausible_pts_C.append(Ctot[i])
        plausible_pts_S.append(Stot[i])

fig = px.scatter_ternary(tot_dissolved_wSmelt, a=plausible_pts_H, b=plausible_pts_C, c=plausible_pts_S)
print("wt% with smelting")
fig.show()
fig.write_image("diss_with_smelting_plausible.svg")

Mol% no smelting


Wt% no smelting


wt% with smelting
